This notebook implements the market simulators for Deep Hedging. Two models are implemented and constrasted: (1) GBM and (2) Heston Market Model

First, the code structure is arranged here in accordance to OOP principles. This will allow easy development of testing and also that of transferring the code to external FASTApi apps. However, jupyter notebooks will be the research base for clear communication and noting of research/design decisions.

In [1]:
import torch
import torch.nn as nn
from abc import ABC, abstractmethod

class MarketSimulator(nn.Module, ABC):
    """
    Abstract Base Class for Market Simulators. 
    To fascilitate interfacing between different models interchangeably with the rest of the system
    
    Output Shape for simulation data: [Batch_Size, Time_Steps, Features]
    """
    def __init__(self, dt, device):
        super().__init__()
        self.dt = dt
        self.device = device

    @abstractmethod
    def forward(self, num_paths, num_steps, initial_prices):
        """
        Generates the market paths.
        
        Args:
            num_paths (int): Number of Monte Carlo simulations. This is the Batch Size of the model.
            num_steps (int): Number of discrete time-steps (T).
            initial_prices (Tensor): Starting prices S_0 [Batch, Assets].
            
        Returns:
            Tensor: The simulated market states.
        """
        pass


The first model: Geometric Brownian Motion (GBM)

This model is defined by the Stochastic Differential Equation: $$dS_t = \mu S_t dt + \sigma S_t dW_t$$

Where, 
* $S_t$: Asset price at time $t$.
* $\mu$: Drift (constant trend of the asset).
* $\sigma$: Volatility (constant standard deviation of returns).
* $dW_t$: A standard Brownian Motion (Wiener Process).

This SDE is discretized as:  $S_{t+\Delta t} = S_t (1 + \mu \Delta t + \sigma \sqrt{\Delta t} Z)$

The above was in price terms, instead of prices themselves log of prices maybe utilized

Log-Price Space ($x_t = \ln S_t$) 

For which, the GBM SDE can be subjected to Itô's Lemma:$$dx_t = \left( \mu - \frac{1}{2}\sigma^2 \right) dt + \sigma dW_t$$
This SDE is discretized as: $x_{t+\Delta t} = x_t + (\mu - 0.5\sigma^2)\Delta t + \sigma \sqrt{\Delta t} Z$

In [3]:
import torch

def pyt_gbm(S0, mu, sigma, T, dt, n_paths, use_log=True):
    steps = int(T / dt)
    n_paths = int(n_paths)
    
    # Generate noise by drawinf from uniform-distribution at [0, 1)
    Z = torch.randn(steps, n_paths)

    # Paths generated in eithe price and log-price  
    if use_log:
        # Log-price: provides numerical stability with non-negative values due to log and also due to additive updates
        drift = (mu - 0.5 * sigma**2) * dt
        diffusion = sigma * torch.sqrt(torch.tensor(dt)) * Z
        log_increments = drift + diffusion
        
        # cumulative sum of log-returns using cumsum
        x = torch.log(torch.tensor(S0)) + torch.cumsum(log_increments, dim=0)
        return torch.exp(x)
    else:
        # Price-space: disadvantaged relative to log-prices due to potential for negative values and also multiplicative updates
        # S_{t+1} = S_t * (1 + mu*dt + sigma*sqrt(dt)*Z)
        increments = 1 + mu * dt + sigma * torch.sqrt(torch.tensor(dt)) * Z
        prices = S0 * torch.cumprod(increments, dim=0) #return values in proces and not in log-prices
        return prices

To test the above method, its compliance to this maybe checked: $E[S_T] = S_0 e^{\mu T}$

In [9]:
# Test for the above method
# To check the compliance of the method with the Expected value, the mean across a large sample is tested

def test_gbm_mean_pytorch():
    S0, mu, sigma, T, dt = 100.0, 0.05, 0.2, 1.0, 0.001
    n_paths = 100000

    # generate price-paths
    prices = pyt_gbm(S0, mu, sigma, T, dt, n_paths)
    terminal_prices = prices[-1] # consider only the final prices
    
    theoretical_mean = S0 * torch.exp(torch.tensor(mu * T)) # value for generated data to be checked against 
    theoretical_mean = theoretical_mean.cpu()
    sample_mean = terminal_prices.mean().item() 
    
    # standard error of the mean = sigma_sample / sqrt(n)
    sem = terminal_prices.std().item() / torch.sqrt(torch.tensor(n_paths))

    # log findings 
    print(f'The Std. Error of the Mean is {sem}')

    # assert within 3 standard errors (99.7% confidence)
    assert abs(sample_mean - theoretical_mean) < 3 * sem

# Perform the test
test_gbm_mean_pytorch()

The Std. Error of the Mean is 0.06728342920541763


In [11]:

class GBMSimulator(MarketSimulator):
    """
    Simulates markets assuming Constant Volatility.
    
    Equation: dS_t = mu * S_t * dt + sigma * S_t * dW_t
    """
    def __init__(self, mu, sigma, dt, device='cpu'):
        super().__init__(dt, device)
        self.mu = mu
        self.sigma = sigma

    def simulate(self, num_paths, num_steps, s0):
        pass


In [10]:
class HestonSimulator(MarketSimulator):
    """
    Simulates markets assuming Stochastic Volatility.
    
    Equation 1 (Price):    dS_t = mu * S_t * dt + sqrt(v_t) * S_t * dW_S
    Equation 2 (Variance): dv_t = kappa * (theta - v_t) * dt + xi * sqrt(v_t) * dW_v
    """
    def __init__(self, mu, kappa, theta, xi, rho, dt, device='cpu'):
        super().__init__(dt, device)
        self.mu = mu
        self.kappa = kappa  # Mean reversion speed
        self.theta = theta  # Long-run variance
        self.xi = xi        # Vol of Vol
        self.rho = rho      # Correlation (Leverage Effect)

    def simulate(self, num_paths, num_steps, s0, v0):
        # todo: Implementation of Euler-Maruyama with Full Truncation -- PyTorch
        pass